In [1]:
!pip install langchain-community langchain-openai  pypdf faiss-cpu tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 25.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.7/298.7 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 35.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 34.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 32.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.7/412.7 kB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.8/50.8 kB 2.8 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.31
    Uninstalling langchain-core-0.3.31:
      Successfully uninstalled langchain-core-0.3.31
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.15
    Uninstalling langchain-0.3.15:
      Successfully uninstalled langchain-0.3.15


In [2]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
import os

In [3]:
from google.colab import userdata
import os


openai_api= userdata.get("OPENAI_API_KEY")

**Uploading PDF Files**
1. one pdf is about mountains
2. the other pdf is about self attention, basically it is attention is all you need reserach paper

In [5]:

pdf_files = ["/content/mountain.pdf", "/content/attention (1).pdf"]


**Load and Split PDF Content**

In [6]:
# Load PDFs
documents = []
for pdf_file in pdf_files:
    loader = PyPDFLoader(pdf_file)
    documents.extend(loader.load())

# Split text into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(documents)

**Create Embeddings and Vector Store**

In [7]:
# Create embeddings
embeddings = OpenAIEmbeddings(model="text-embedding-3-small",openai_api_key=openai_api)

# Create vector store
vector_store = FAISS.from_documents(texts, embeddings)

<ipython-input-7-836a2c5e81d2>:2: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings(model="text-embedding-3-small",openai_api_key=openai_api)


**Set Up the Chatbot**

**Use LangChain's RetrievalQA to create a question-answering chain.**

In [9]:

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0,openai_api_key=openai_api)

# Create the QA chain
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vector_store.as_retriever()
)

**Ask Questions**

**Now you can ask questions based on the content of the PDFs.**

In [10]:
while True:
    query = input("You: ")
    if query.lower() in ["exit", "quit"]:
        break
    response = qa_chain.run(query)
    print(f"Bot: {response}")

You: mountains in america


<ipython-input-10-6b2db06d2e7b>:5: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = qa_chain.run(query)


Bot: In North America, two major mountain ranges are the Appalachians and the Rockies. The Appalachians are the oldest mountain chain in North America, stretching from Newfoundland to central Alabama. The Rockies extend for more than three thousand miles from Alaska to New Mexico. The highest point in North America is Denali in Alaska. The major pass through the Rockies for travelers in the 1800s was South Pass in Wyoming.
You: highest mountain in world
Bot: The highest mountain in the world is Mount Everest, located in the Great Himalayas. It has an elevation of 29,028 feet (8,848 meters).
You: self attention is what
Bot: Self-attention is an attention mechanism that relates different positions of a single sequence in order to compute a representation of the sequence. It has been successfully used in various tasks such as reading comprehension, abstractive summarization, textual entailment, and learning task-independent sentence representations.
You: waht is position enscodng
Bot: Pos